In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("/content/kg_dataset (1).csv")

In [3]:
df = df.dropna().drop_duplicates()

In [4]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  #many spaces
    text = text.strip()               #spaces at start and end
    return text

df["Text"] = df["Text"].apply(clean_text)
df["headline"] = df["headline"].apply(clean_text)

In [5]:
MAX_LEN = 512
df = df[df["Text"].str.len() < MAX_LEN]

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["Text"].tolist(),
    df["headline"].tolist(),
    test_size=0.1,
    random_state=42
)

print("Train size:", len(train_texts))
print("Validation size:", len(val_texts))

Train size: 1555
Validation size: 173


In [7]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

#model
model_name = "google/mt5-small"

tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be us

In [8]:
#max input characters and max output
MAX_INPUT_LENGTH = 512
MAX_TARGET_LENGTH = 52

#training tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_INPUT_LENGTH)
train_targets = tokenizer(train_labels, truncation=True, padding=True, max_length=MAX_TARGET_LENGTH)

print(train_texts[:5])
print(train_labels[:5])

for i in range(5):
    print("LABEL", i, ":", tokenizer.decode(train_targets["input_ids"][i]))

#validation tokenization
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_INPUT_LENGTH)
val_targets = tokenizer(val_labels, truncation=True, padding=True, max_length=MAX_TARGET_LENGTH)

['Бүгүн,5-декабрда Жогорку КеңешРавшанбек Сабировдуэмгек, социалдык коргоо жана миграция министри кызматына дайындоого макулдук берген. Депутаттар анын талапкерлигине талкуусуз добуш беришти. 77 депутат «макул» деп добуш берди. Жалпы 83 депутат катталган. Кечээ, 4-декабрдаЖылдыз Полотоваэмгек, социалдык камсыздоо жана миграция министри кызматынан бошотулду.Равшанбек Сабировэмгек, социалдык камсыздоо жана миграция министринин милдетин аткаруучу болуп дайындалган.', 'Билим берүү жана илим министрлиги мектептер үчүн мамлекеттик электрондук күндөлүктү түзүү иштери аяктап калганын билдирди. Министрлик тарабынан иштелип чыккан электрондук күндөлүк окуучулар жана алардын ата-энелери, мектептер үчүн акысыз болот.', 'Бүгүн,27-августта окумуштуу, академикИлгиз Төрөкулович Айтматовдүйнөдөн кайтты. Бул тууралуу анын кызыЖамиля Айтматовабилдирди. Илгиз Айтматов— улуу жазуучуЧыңгыз Айтматовдуниниси. Ал 1931-жылы 8-февралда Фрунзе шаарында туулган. Көп жыл бою Кыргыз улуттук академиясында эмгектенген

In [9]:
class KyrgyzHeadlineDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, targets):
        self.encodings = encodings
        self.targets = targets

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        labels = self.targets["input_ids"][idx]
        #all pads to -100
        labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]

        item = {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(labels),
        }
        return item

In [10]:
train_dataset = KyrgyzHeadlineDataset(train_encodings, train_targets)
val_dataset = KyrgyzHeadlineDataset(val_encodings, val_targets)

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=100,
    do_eval=True,
    save_total_limit=2,
    predict_with_generate=True,
)

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [14]:
!pip install rouge_score

In [15]:
from datasets import load_metric

rouge_metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure,
    }

<ipython-input-15-093e4c08b2a2>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


In [16]:
import torch
print(torch.cuda.is_available())

True


In [19]:
import shutil
shutil.rmtree('./logs', ignore_errors=True)

In [22]:
sample = train_dataset[0]
print(sample['labels'])

tensor([  1580, 152136,   3591,  10185, 101583,  33385,  13390, 166969,  33197,
         11840,   8662,  40118, 170087,  39439,   1625,  76745,    686,  30513,
        186153,      1,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100])


In [23]:
print("input_ids:", tokenizer.decode(sample["input_ids"]))
print("labels   :", tokenizer.decode([token if token != -100 else tokenizer.pad_token_id for token in sample["labels"]]))

input_ids: Бүгүн,5-декабрда Жогорку КеңешРавшанбек Сабировдуэмгек, социалдык коргоо жана миграция министри кызматына дайындоого макулдук берген. Депутаттар анын талапкерлигине талкуусуз добуш беришти. 77 депутат «макул» деп добуш берди. Жалпы 83 депутат катталган. Кечээ, 4-декабрдаЖылдыз Полотоваэмгек, социалдык камсыздоо жана миграция министри кызматынан бошотулду.Равшанбек Сабировэмгек, социалдык камсыздоо жана миграция министринин милдетин аткаруучу болуп дайындалган.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
labels   : Депутаттар эмгек министрине Равшанбек Сабировдун талапкерлигин жактырды</s><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [24]:
labels_for_decode = [token if token != -100 else tokenizer.pad_token_id for token in sample["labels"]]
print(tokenizer.decode(labels_for_decode))

Депутаттар эмгек министрине Равшанбек Сабировдун талапкерлигин жактырды</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [25]:
print("Train dataset size:", len(train_dataset))

Train dataset size: 1555


In [30]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=100,
    do_eval=True,
    save_total_limit=2,
    predict_with_generate=True,
    report_to="none", #wandb off
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

<ipython-input-30-227ada6fca7b>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
50,19.115800
100,11.924400
150,8.888400
200,6.894000
250,5.814400
300,4.982300
350,4.785400
400,4.426900
450,4.322000
500,4.212200


TrainOutput(global_step=585, training_loss=7.059413055680756, metrics={'train_runtime': 596.3621, 'train_samples_per_second': 7.822, 'train_steps_per_second': 0.981, 'total_flos': 1088780296089600.0, 'train_loss': 7.059413055680756, 'epoch': 3.0})

In [31]:
trainer.save_model("headline_model")
tokenizer.save_pretrained("headline_model")

('headline_model/tokenizer_config.json',
 'headline_model/special_tokens_map.json',
 'headline_model/spiece.model',
 'headline_model/added_tokens.json')

In [40]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("headline_model").to("cuda")
tokenizer = MT5Tokenizer.from_pretrained("headline_model")

text = "Бүгүн эртең менен жаан жаап баштады, азыр жаабай калды, күн чыгып кайра ысык болуп калды"

# Указываем max_length явно
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to("cuda")

# Генерация заголовка
summary = model.generate(
    **inputs,
    max_length=64,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(summary[0], skip_special_tokens=True))

<extra_id_0> жаап баштады


In [35]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 2.914217948913574, 'eval_runtime': 1.6819, 'eval_samples_per_second': 102.861, 'eval_steps_per_second': 13.081, 'epoch': 3.0}
